In [56]:
from importlib import reload
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='whitegrid')
import ard_logging
import logging
ard_logging.init(force_debug=True)
LOG = logging.getLogger(__name__)
LOG.info('Hello from notebook')

2022-06-19 22:35:09,659:    INFO:      ard_logging: MainProcess 2188: Logging initialized.
2022-06-19 22:35:09,662:    INFO:         __main__: MainProcess 2188: Hello from notebook


In [18]:
df = sns.load_dataset('diamonds')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [73]:
from __future__ import annotations

from attrs import define

@define
class hp:
    name: str
    
    @staticmethod
    def resolve(v, bindings):
        if isinstance(v, hp):
            return bindings[v.name]
        return v

class DeMean:
    """
    Two operations: fit and apply. Fit-time and apply-time.

    Params and hyperparams.
    """
    def __init__(self, cols):
        self.cols = cols

    def fit(
        self: DeMean,
        X_fit: pd.DataFrame,
        bindings: Optional[dict[str, object]]=None
    ) -> FitDeMean:
        LOG.debug('Fitting Transform on %d rows of data: %r', len(X_fit), self)
        return self.FitDeMean(self, bindings or {}, X_fit)

    class FitDeMean:
        def __init__(self, tf, bindings, X_fit):
            self.cols = hp.resolve(tf.cols, bindings)
            self.means = X_fit[self.cols].mean()

        def apply(self, X_apply: pd.DataFrame) -> pd.DataFrame:
            LOG.debug('Applying Transform to %d rows of data: %r', len(X_apply),
                self)
            means = self.means
            return X_apply.assign(**{
                c: X_apply[c] - means[c]
                for c in self.cols
            })


In [74]:
t = DeMean(['price'])
t = DeMean(hp('features'))
#t.fit(df)
#t.apply(df)
print(t.cols)
t.fit(df, bindings={'features': ['price']}).apply(df).head()

2022-06-19 22:42:37,591:   DEBUG:         __main__: MainProcess 2188: Fitting Transform on 53940 rows of data: <__main__.DeMean object at 0x7f415f25dd60>
2022-06-19 22:42:37,609:   DEBUG:         __main__: MainProcess 2188: Applying Transform to 53940 rows of data: <__main__.DeMean.FitDeMean object at 0x7f415f25d8b0>


hp(name='features')


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,-3606.799722,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,-3606.799722,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,-3605.799722,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,-3598.799722,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,-3597.799722,4.34,4.35,2.75


In [54]:
tfit = t.fit(df, bindings={'features': ['price']})
tfit.cols, tfit.means

(['price'],
 price    3932.799722
 dtype: float64)

In [266]:
from abc import ABC, abstractmethod
from functools import wraps

class Transform(ABC):
    "The abstract base class of all Transforms"
    def __init_subclass__(cls, /, no_magic=False, **kwargs):
        super().__init_subclass__(**kwargs)
        if no_magic:
            return
        class_name = cls.__name__
        cls.fit_class_name = f'Fit{class_name}'
        class DerivedFitTransform(FitTransform):
            _apply = cls._apply
            
        fit_class = DerivedFitTransform
        fit_class.__name__ = cls.fit_class_name
        fit_class.__qualname__ = '.'.join((cls.__qualname__, cls.fit_class_name))
        fit_class.__doc__ = FitTransform.__doc__.format(
            class_name=class_name)
        cls.fit.__annotations__['return'] = fit_class.__qualname__
        setattr(cls, cls.fit_class_name, fit_class)

    def __init__(self, cols):
        "Transform init docstr"
        self.cols = cols

    @abstractmethod
    def _fit(self, X_fit):
        raise NotImplementedError
        
    @abstractmethod
    def _apply(self, X_apply, state=None):
        raise NotImplementedError
        
    fit_class_name = FitTransform.__name__
        
    def fit(
        self,
        X_fit: pd.DataFrame,
        bindings: Optional[dict[str, object]]=None
    ) -> FitTransform:
        LOG.debug('Fitting Transform on %d rows of data: %r', len(X_fit), self)
        fit_class = getattr(self, self.fit_class_name)
        #return fit_class(self, X_fit, bindings)
        return fit_class.from_fitting(self, X_fit, bindings)

class FitTransform(ABC):
    "The result of fitting a {class_name} Transform. Blah blah."
    
    def __init__(self, transform, X_fit, bindings=None):
        "Docstr for FitTransform.__init__"
        self.cols = hp.resolve(transform.cols, bindings or {})
        self.__tf = transform
        self.__state = transform._fit.__func__(self, X_fit)
    
    @classmethod
    def from_fitting(cls, transform, X_fit, bindings=None):
        # todo: 
        return cls(transform, X_fit, bindings)
        
    @abstractmethod
    def _apply(self, X_apply, state=None):
        raise NotImplementedError
        
    def apply(self, X_apply: pd.DataFrame) -> pd.DataFrame:
        "Docstr for FitTransform.apply"
        LOG.debug('Applying Transform to %d rows of data: %r', len(X_apply),
            self)
        return self._apply(X_apply, state=self.__state)
    
    # TODO: refit()
    
    def original(self):
        # what if there's a field named 'original'?
        return self.__tf
    
    def state(self):
        # what if there's a field named 'state'?
        return self.__state
        
class DeMean(Transform):
    "Docstr for MyDeMean"
    
    def _fit(self, X_fit):
        return X_fit[self.cols].mean()
    
    def _apply(self, X_apply, state):
        means = state
        return X_apply.assign(**{
            c: X_apply[c] - means[c]
            for c in self.cols
        })
        

t = DeMean(cols=['price'])
tfit = t.fit(df)
tfit.apply(df).head()

2022-06-20 00:25:12,742:   DEBUG:         __main__: MainProcess 2188: Fitting Transform on 53940 rows of data: <__main__.DeMean object at 0x7f415de44730>
2022-06-20 00:25:12,750:   DEBUG:         __main__: MainProcess 2188: Applying Transform to 53940 rows of data: <__main__.DeMean.FitDeMean object at 0x7f415e8bdc70>


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,-3606.799722,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,-3606.799722,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,-3605.799722,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,-3598.799722,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,-3597.799722,4.34,4.35,2.75


In [245]:
DeMean?

Init signature: DeMean(cols)
Docstring:      Docstr for MyDeMean
Init docstring: Transform init docstr
File:           /usr/lib/python3.9/abc.py
Type:           ABCMeta
Subclasses:     


In [214]:
(DeMean.__module__,
DeMean.__name__,
DeMean.__qualname__)

('__main__', 'DeMean', 'DeMean')

In [215]:
(DeMean.FitDeMean.__module__,
DeMean.FitDeMean.__name__,
DeMean.FitDeMean.__qualname__)

('__main__', 'FitDeMean', 'DeMean.FitDeMean')

In [250]:
DeMean.fit.__annotations__

{'X_fit': 'pd.DataFrame',
 'bindings': 'Optional[dict[str, object]]',
 'return': 'DeMean.FitDeMean'}

In [256]:
help(DeMean)

Help on class DeMean in module __main__:

class DeMean(Transform)
 |  DeMean(cols)
 |  
 |  Docstr for MyDeMean
 |  
 |  Method resolution order:
 |      DeMean
 |      Transform
 |      abc.ABC
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  FitDeMean = <class '__main__.DeMean.FitDeMean'>
 |      The result of fitting a DeMean Transform. Blah blah.
 |  
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  fit_class_name = 'FitDeMean'
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Transform:
 |  
 |  __init__(self, cols)
 |      Transform init docstr
 |  
 |  fit(self, X_fit: 'pd.DataFrame', bindings: 'Optional[dict[str, object]]' = None) -> 'DeMean.FitDeMean'
 |  
 |  ----------------------------------------------------------------------
 |  Class methods inherited from Transform:
 |  
 |  __init_subclass__(no_magic=False, **kwargs) from abc.ABCMeta
 |      This method is called when a class i

In [257]:
help(DeMean.FitDeMean)

Help on class FitDeMean in module __main__:

class FitDeMean(FitTransform)
 |  FitDeMean(transform, X_fit, bindings=None)
 |  
 |  The result of fitting a DeMean Transform. Blah blah.
 |  
 |  Method resolution order:
 |      FitDeMean
 |      FitTransform
 |      abc.ABC
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from FitTransform:
 |  
 |  __init__(self, transform, X_fit, bindings=None)
 |      Docstr for FitTransform.__init__
 |  
 |  apply(self, X_apply: 'pd.DataFrame') -> 'pd.DataFrame'
 |      Docstr for FitTransform.apply
 |  
 |  from_fitting(cls, transform, X_fit, bindings=None)
 |  
 |  original(self)
 |  
 |  state(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from FitTransform:
 |  
 |  __dict__
 |      dictionary for instance varia

In [231]:
t

In [232]:
tfit

In [233]:
tfit.__class__.__name__

'FitDeMean'

In [234]:
DeMean.FitDeMean?

Init signature: DeMean.FitDeMean(tf, bindings, X_fit)
Docstring:      The result of fitting a DeMean. Blah blah.
Init docstring: Docstr for FitTransform.__init__
File:           /usr/lib/python3.9/abc.py
Type:           ABCMeta
Subclasses:     


In [236]:
tfit.apply?

Signature: tfit.apply(X_apply: 'pd.DataFrame') -> 'pd.DataFrame'
Docstring: Docstr for FitTransform.apply
File:      /tmp/ipykernel_2188/2031211177.py
Type:      method


In [265]:
DeMean.FitDeMean.from_fitting(t, df, {})

In [281]:
class StatelessTransform(Transform):
    def _fit(self, X_fit):
        return None
    
    #@abstractmethod
    #def _apply(self, X_apply):
    #    # no kwarg for state
    #    raise NotImplementedError
    
    #def apply(self, X_apply: pd.DataFrame) -> pd.DataFrame:
    #    pass

In [282]:
StatelessTransform(cols=['price'])

TypeError: Can't instantiate abstract class StatelessTransform with abstract method _apply

In [285]:
class AssignConstant(StatelessTransform):
    def __init__(self, cols, value):
        super().__init__(cols)
        self.value = value
        
    def _apply(self, X_apply, state=None):
        return X_apply.assign(**{
            c: self.value
            for c in self.cols
        })
        
        

In [286]:
AssignConstant(['price'], 1).fit(df).apply(df)

2022-06-20 00:38:02,290:   DEBUG:         __main__: MainProcess 2188: Fitting Transform on 53940 rows of data: <__main__.AssignConstant object at 0x7f415dd3a2e0>
2022-06-20 00:38:02,293:   DEBUG:         __main__: MainProcess 2188: Applying Transform to 53940 rows of data: <__main__.AssignConstant.FitAssignConstant object at 0x7f415e018670>


AttributeError: 'FitAssignConstant' object has no attribute 'value'